In [1]:
import sys
sys.path.append('/home/samer/projects/fuzzy_sql/src') #This will enable reading the modules
from fuzzy_sql.fuzzy_sql import *

In [2]:
def _assign_dtype(df, dict):
    #Correct dtypes of real and syn dataframes before saving in the database 
    #map metaddata into dtype dict with pandas dtypes
    #cols in df shall match keys in in_dict
    assert bool(set(df.columns).intersection(set(dict.keys())))
    out_dict={}
    for key in dict:
        if dict[key] in ['quantitative','continuous','interval','ratio']:
            out_dict[key]='float64'
        elif dict[key] in ['date','time','datetime']:
            out_dict[key]='datetime64'
        else:
            out_dict[key]='category'
    
    for col in df.columns:
        df[col]=df[col].astype(out_dict[col])

    return df


In [3]:
class LONG_QUERY():
    """ Generates random queries for baseline-longitudinal datasets. 
    """

    def __init__(self, db_conn: object, parent_tbl_name: str, child_tbl_name: str, metadata: dict,params: dict , seed=False):
        """ 
        Args:
            db_conn: The connection object of the sqlite database where the data exists.
            parent_tbl_name: The name of the parent table (i.e. baseline data) in the database.
            child_tbl_name: The name of the child table (i.e. longitudinal data) in the database.
            metadata: A dictionary that includes table's variable names (i.e. column names) as keys and types of variables as values. THey types shall be restricted to: 'continuous', 'data' and 'nominal'. Any table shall have at least one nominal variable.
            params: A dictionary that includes the set of parameters that are necessary for generating the random queries. 
            seed: If set to True, generated random queries become deterministic. 
        """
        self.SEED=seed
        self.seed_no=141

        self.CUR = db_conn.cursor()
        self.PARENT_NAME = parent_tbl_name #RP = Real Parent
        self.CHILD_NAME = child_tbl_name #RC = Real Child
        self.metadata=copy.deepcopy(metadata)
        
        #Fetch Real data (both parent and child)
        self.PARENT_DF=pd.read_sql_query(f'SELECT * FROM {self.PARENT_NAME}', db_conn) #Real Parent Dataframe
        self.CHILD_DF=pd.read_sql_query(f'SELECT * FROM {self.CHILD_NAME}', db_conn) #Real Child Dataframe


        #Get foreign key name
        self.FKEY_NAME=self.metadata['key']

        #Delete foreign key from child variables to avoid repetition of variable in various expression
        del self.metadata['child'][self.FKEY_NAME]


        #Segregate variables into lists based on their types
        self.CAT_VARS={} #Parent Categorical Variables
        self.CNT_VARS={}
        self.DT_VARS={}
        self.CAT_VARS['parent']=[key for key, value in self.metadata['parent'].items() if value in ['qualitative','categorical','nominal','discrete','ordinal','dichotomous']]
        self.CAT_VARS['child']=[key for key, value in self.metadata['child'].items() if value in ['qualitative','categorical','nominal','discrete','ordinal','dichotomous']]
        self.CNT_VARS['parent']=[key for key, value in self.metadata['parent'].items() if value in ['quantitative','continuous','interval','ratio']]
        self.CNT_VARS['child']=[key for key, value in self.metadata['child'].items() if value in ['quantitative','continuous','interval','ratio']]
        self.DT_VARS['parent']=[key for key, value in self.metadata['parent'].items() if value in ['date','time','datetime']]
        self.DT_VARS['child']=[key for key, value in self.metadata['child'].items() if value in ['date','time','datetime']]



        # Aggregate function applies only when there is at least one continuous variable 
        self.AGG_FNCTN=True if len(self.CNT_VARS['parent'])!=0 or len(self.CNT_VARS['child'])!=0 else False

        # Define random query attributes
        self.ATTRS=params #General attributes that can be set by the user


        # Generate dictionaries of bags for various variables
        self.CAT_VAL_BAGS={}
        self.CNT_VAL_BAGS={}
        self.DT_VAL_BAGS={}
        self.CAT_VAL_BAGS['parent']=self._make_bags(self.PARENT_DF[self.CAT_VARS['parent']])
        self.CNT_VAL_BAGS['parent']=self._make_bags(self.PARENT_DF[self.CNT_VARS['parent']])
        self.DT_VAL_BAGS['parent']=self._make_bags(self.PARENT_DF[self.DT_VARS['parent']])
        self.CAT_VAL_BAGS['child']=self._make_bags(self.CHILD_DF[self.CAT_VARS['child']])
        self.CNT_VAL_BAGS['child']=self._make_bags(self.CHILD_DF[self.CNT_VARS['child']])
        self.DT_VAL_BAGS['child']=self._make_bags(self.CHILD_DF[self.DT_VARS['child']])
        

    def _make_bags(self,df:pd.DataFrame)-> dict:
        val_bags={}
        for var in df.columns:
            vals=df[var].values
            vals=[x for x in vals if x==x] #drop nan
            vals=list(filter(None, vals)) #drop None
            val_bags[var]=vals if len(vals)!=0 else ['N/A']
        return val_bags

    def make_query(self,cur: object, query_exp: str)-> pd.DataFrame:
        cur.execute(query_exp)
        query = cur.fetchall()
        query=pd.DataFrame(query, columns=[description[0] for description in cur.description])
        return query

    # def _get_var_idx(self, var_name):
    #     if var_name in self.PARENT_DF.columns: #search in parent
    #         idx=self.PARENT_DF.columns.get_loc(var_name)
    #         return 'parent',idx
    #     elif var_name in self.CHILD_DF.columns: #search in child
    #         idx=self.CHILD_DF.columns.get_loc(var_name)
    #         return 'child', idx
    #     else:
    #         raise Exception(f"{var_name} not found in parent or child tables!")

    def _mix_vars(self,*args):
        #accepts variable length of arguments as tuples where each tuple consists of the table name and the some variables that belong to that table
        # returns mixed variables in one list but each variable is concatenated with its respective table name
        mixed_vars=[]
        for arg in args:
            vars=[arg[0]+'.'+x for x in arg[1]]
            mixed_vars+=vars
        return mixed_vars

    def _get_twin_lst(self, in_lst,in_parent_name, out_parent_name, in_child_name, out_child_name)-> list:
        out_lst=[var.replace(in_parent_name,out_parent_name ) for var in in_lst] 
        out_lst=[var.replace(in_child_name,out_child_name ) for var in out_lst] 
        return out_lst

    def _get_rnd_groupby_lst(self)-> list:
        #You can group by CAT_VARS whether from parent or child or both
        #returned picked cat vars include the concatenated table name
        if self.SEED:
            np.random.seed(self.seed_no)
            random.seed(self.seed_no)
        all_cat_vars=self._mix_vars((self.PARENT_NAME,self.CAT_VARS['parent']),(self.CHILD_NAME,self.CAT_VARS['child']))
        # parent_cat_vars=[self.PARENT_NAME+'.'+x for x in self.CAT_VARS['parent']]
        # child_cat_vars=[self.CHILD_NAME+'.'+x for x in self.CAT_VARS['child']]
        # all_cat_vars=parent_cat_vars +child_cat_vars
        n_vars_bag=np.arange(1, 1+len(all_cat_vars)) 
        if self.ATTRS['LESS_GRP_VARS']:#define slope-down discrete distribution 
            n_var_probs=n_vars_bag[::-1]/n_vars_bag.sum()
            # n_var_probs= np.zeros_like(n_vars_bag)
            # n_var_probs[0]=1
            n_vars=np.random.choice(n_vars_bag, p=n_var_probs)
        else:
            n_vars=np.random.choice(n_vars_bag)
        picked_vars = random.sample(all_cat_vars, n_vars)
        return picked_vars


    def _get_rnd_aggfntn_tpl(self) -> tuple:
        #returns a random tuple of agg function and continuous OR date variable 
        # continuous variable can be from either parent or child tables 
        if self.SEED:
            np.random.seed(self.seed_no)
            random.seed(self.seed_no)
        all_possible_vars=self._mix_vars((self.PARENT_NAME,self.CNT_VARS['parent']),(self.PARENT_NAME,self.DT_VARS['parent']),(self.CHILD_NAME,self.CNT_VARS['child']),(self.CHILD_NAME,self.DT_VARS['child']))
        picked_var=np.random.choice(all_possible_vars)
        picked_op=np.random.choice(list(self.ATTRS['AGG_OPS'].keys()), p=list(self.ATTRS['AGG_OPS'].values()))
        return (picked_op,picked_var)




    




#########################################################################

    def _build_agg_expr(self,  pname: str, cname: str, fkey: str, groupby_lst: list) -> str:
        expr1=f'SELECT *,COUNT(*) FROM {pname} JOIN {cname} ON {pname}.{fkey} = {cname}.{fkey}'
        expr2_1=' GROUP BY '
        expr2_2=f'{groupby_lst}'
        expr2_2=expr2_2.replace("[","")
        expr2_2=expr2_2.replace("]","")
        expr2_2=expr2_2.replace("'","")
        return expr1+expr2_1+expr2_2

    
    def make_single_agg_query(self) -> dict:
        dic={}
        single_grp_lst=self._get_rnd_groupby_lst()
        single_expr=self._build_agg_expr(self.PARENT_NAME, self.CHILD_NAME,self.FKEY_NAME, single_grp_lst)
        query=self.make_query(self.CUR, single_expr)
        dic['query']=query
        dic['query_desc']={
            "type":"single_agg",
            "sql":single_expr,
            "n_rows":query.shape[0],
            "n_cols":query.shape[1]
        }
        return dic

    def make_twin_agg_query(self, twin_parent_name, twin_child_name):
        dic={}
        real_grp_lst =self._get_rnd_groupby_lst()
        syn_grp_lst=self._get_twin_lst(real_grp_lst, self.PARENT_NAME,twin_parent_name, self.CHILD_NAME,twin_child_name)
        real_expr=self._build_agg_expr(self.PARENT_NAME, self.CHILD_NAME,self.FKEY_NAME, real_grp_lst)
        syn_expr=self._build_agg_expr(twin_parent_name, twin_child_name,self.FKEY_NAME, syn_grp_lst)
        query_real=self.make_query(self.CUR, real_expr)
        query_syn=self.make_query(self.CUR, syn_expr)
        dic['query_real']=query_real
        dic['query_syn']=query_syn
        dic['query_desc']={
            "type":"twin_agg",
            "sql_real":real_expr,
            "n_cols_real":query_real.shape[1],
            "n_rows_real":query_real.shape[0],
            "sql_syn":syn_expr,
            "n_cols_syn":query_syn.shape[1],
            "n_rows_syn":query_syn.shape[0],
        }
        return dic

     

    def _build_agg_expr_w_aggfntn(self,pname: str, cname: str, fkey: str, agg_fntn_tpl: tuple, groupby_lst: list) -> str:
        fkey=self.FKEY_NAME
        expr1=f'SELECT *,COUNT(*), {agg_fntn_tpl[0]}({agg_fntn_tpl[1]}) FROM {pname} JOIN {cname} ON {pname}.{fkey} = {cname}.{fkey}'
        expr2_1=' GROUP BY '
        expr2_2=f'{groupby_lst}'
        expr2_2=expr2_2.replace("[","")
        expr2_2=expr2_2.replace("]","")
        expr2_2=expr2_2.replace("'","")
        expr=expr1+expr2_1+expr2_2
        return expr
    

    def make_single_agg_query_w_aggfntn(self):
        dic={}
        single_grp_lst=self._get_rnd_groupby_lst()
        agg_fntn_tpl=self._get_rnd_aggfntn_tpl()
        expr=self._build_agg_expr_w_aggfntn(self.PARENT_NAME,self.CHILD_NAME, self.FKEY_NAME,agg_fntn_tpl,single_grp_lst)
        query=self.make_query(self.CUR, expr)
        dic['query']=query
        dic['query_desc']={
            "type":"single_agg",
            "sql":expr,
            "n_rows":query.shape[0],
            "n_cols":query.shape[1]
        }
        return dic

    def make_twin_agg_query_w_aggfntn(self,twintbl_parent_name,twintbl_child_name):
        dic={}
        real_groupby_lst=self._get_rnd_groupby_lst()
        syn_groupby_lst=self._get_twin_lst(real_groupby_lst, self.PARENT_NAME,twintbl_parent_name, self.CHILD_NAME, twintbl_child_name)
        real_aggfntn_tpl=self._get_rnd_aggfntn_tpl()
        syn_aggfntn_tpl=self._get_twin_lst(real_aggfntn_tpl, self.PARENT_NAME,twintbl_parent_name, self.CHILD_NAME, twintbl_child_name)
        real_expr=self._build_agg_expr_w_aggfntn(self.PARENT_NAME,self.CHILD_NAME,self.FKEY_NAME,real_aggfntn_tpl, real_groupby_lst)
        syn_expr=self._build_agg_expr_w_aggfntn(twintbl_parent_name,twintbl_child_name,self.FKEY_NAME,syn_aggfntn_tpl, syn_groupby_lst)
        query_real=self.make_query(self.CUR, real_expr)
        query_syn=self.make_query(self.CUR, syn_expr)
        dic['query_real']=query_real
        dic['query_syn']=query_syn
        dic['query_desc']={
            "type":"twin_agg",
            "sql_real":real_expr,
            "n_cols_real":query_real.shape[1],
            "n_rows_real":query_real.shape[0],
            "sql_syn":syn_expr,
            "n_cols_syn":query_syn.shape[1],
            "n_rows_syn":query_syn.shape[0],
            }
        return dic

# ######################################################################################
    
#     def _build_flter_single_expr(self, val_cmp_term: str) -> str:
#         pass


#     def _build_aggfltr_single_expr(self, val_cmp_term: str, groupby_term: str):
#         pass

#     def _build_aggfltr_w_fntn_single_expr(self, agg_fnt_term: str, val_cmp_term, groupby_term: str):
#         pass

#     def _build_flter_twin_expr(self, val_cmp_term: str) -> str:
#         pass

#     def _build_agg_twin_expr(self,groupby_term: str) -> str:
#         pass

#     def _build_agg_w_fntn_twin_expr(self, agg_fnt_term: str, groupby_term: str):
#         pass

#     def _build_aggfltr_twin_expr(self, val_cmp_term: str, groupby_term: str):
#         pass

#     def _build_aggfltr_w_fntn_twin_expr(self, agg_fnt_term: str, val_cmp_term, groupby_term: str):
#         pass




        


        




In [4]:
rp_path="/home/samer/projects/fuzzy_sql/data/longitudinal/ready/real/b_sample.csv" #real parent (baseline) path 
rc_path="/home/samer/projects/fuzzy_sql/data/longitudinal/ready/real/l_sample.csv" #real child (longitudinal) path 
sp_path="/home/samer/projects/fuzzy_sql/data/longitudinal/ready/synthetic/b_sample_syn.csv" #synthetic parent (baseline) path 
sc_path="/home/samer/projects/fuzzy_sql/data/longitudinal/ready/synthetic/l_sample_syn.csv" #synthetic child (longitudinal) path 
meta_path="/home/samer/projects/fuzzy_sql/data/longitudinal/ready/metadata/sample.json" #metdata path

In [5]:
# read data frames with all variables read as string and eliminate the apostrophe  '
rp=load_csv(rp_path) 
rc=load_csv(rc_path) 
sp=load_csv(sp_path)  
sc=load_csv(sc_path) 
import _json
with open(meta_path) as f:
    meta=json.load(f) #metadata for the data 

In [6]:
#fix datatypes in loaded csv 
rp=_assign_dtype(rp, meta['parent'])
rc=_assign_dtype(rc, meta['child'])
sp=_assign_dtype(sp, meta['parent'])
sc=_assign_dtype(sc, meta['child'])

In [7]:
#define default parameters
DFLT_PARAMS={
    'AGG_OPS':{'AVG':0.5, 'SUM':0.3, 'MAX':0.1, 'MIN':0.1 },
    'LOGIC_OPS':{'AND':0.5,'OR':0.5},
    'NOT_STATE':{'0':1, '1':0},
    'CAT_OPS':{'=':0.25, '<>':0.25, 'LIKE':0.25, 'IN':0.25},
    'CNT_OPS':{'=':0.2, '>':0.1, '<':0.1, '>=':0.1, '<=':0.1, '<>':0.1, 'BETWEEN':0.3},
    'DT_OPS':{'=':0.2, '>':0.1, '<':0.1, '>=':0, '<=':0, '<>':0.1, 'BETWEEN':0.3, 'IN':0.2},
    'LESS_GRP_VARS': False, # enforce bias in random queries toward smaller number of groupby vars. Default is no bias (i.e. uniform sampling)
    'LESS_CMP_VARS':False, # enforce bias in random queries toward small number of  comparison terms. Default is no bias (i.e. uniform sampling)
}

In [8]:
# import real data into database
conn = sqlite3.connect('fuzzy_sql.db')
make_table('sample_r_b', rp, conn)
make_table('sample_r_l', rc, conn)
make_table('sample_s_b', sp, conn)
make_table('sample_s_l', sc, conn)



Table sample_r_b already exists in the database
Table sample_r_l already exists in the database
Table sample_s_b already exists in the database
Table sample_s_l already exists in the database


In [9]:
self=LONG_QUERY(conn,'sample_r_b','sample_r_l', meta,DFLT_PARAMS)

In [10]:
self.make_single_agg_query()

{'query':       PNUM_R   AGE DIED DNR FEMALE HOSPBRTH Homeless HISPANIC NEOMAT PL_CBSA  \
 0    1362038  22.0    0   0   None        0        0     None      0    None   
 1     482884   NaN    0   0   None        0        0     None      1       0   
 2     207404  72.0    0   0   None        0        0     None      0       0   
 3      63394  26.0    0   0   None        0        0     None      0       0   
 4      82222   NaN    0   0   None        0        0     None      1       1   
 5     948920  17.0    0   0   None        0        0     None      0       1   
 6   14089895  26.0    0   0   None        0        0     None      1       1   
 7     418860  77.0    0   0   None        0        0     None      0       1   
 8    7604273  57.0    0   0   None        0        0     None      0       1   
 9       6116  12.0    0   0   None        0        0     None      0       2   
 10     73430  49.0    0   0   None        0        0     None      0       2   
 11     67949  62.0

In [11]:
self.make_twin_agg_query('sample_s_b','sample_s_l')

{'query_real':        PNUM_R   AGE DIED DNR FEMALE HOSPBRTH Homeless HISPANIC NEOMAT PL_CBSA  \
 0        5471  60.0    0   0      1        0        0        2      0    None   
 1        6407  43.0    0   0      1        0        0        2      0       2   
 2     2984885  44.0    0   0      0        0        0        2      0       2   
 3         183  50.0    0   0      1        0        0        2      0       2   
 4       13024  77.0    0   0      0        0        0        2      0       2   
 ..        ...   ...  ...  ..    ...      ...      ...      ...    ...     ...   
 181   4668156  68.0    0   0      0        0        0        2      0       0   
 182   1518659  39.0    0   0      0        0        0        2      0       0   
 183  14121079  56.0    0   0      0        0        0        2      0       0   
 184  14155990   0.0    0   0      1        1        0        1      2       2   
 185   3387011   0.0    0   0      1        1        0        2      2       2   
 


In [13]:
self.make_single_agg_query_w_aggfntn()

{'query':           PNUM_R   AGE DIED DNR FEMALE HOSPBRTH Homeless HISPANIC NEOMAT  \
 0       10044368  47.0    0   0      0        0        0     None      0   
 1       10044368  47.0    0   0      0        0        0     None      0   
 2        1018764  52.0    0   0      1        0        0        2      0   
 3        1018764  52.0    0   0      1        0        0        2      0   
 4       10235227  52.0    0   0      1        0        0        2      0   
 ...          ...   ...  ...  ..    ...      ...      ...      ...    ...   
 457222   9963822  91.0    0   0      1        0        0        2      0   
 457223     99676  77.0    0   0      0        0        0        2      0   
 457224     99676  77.0    0   0      0        0        0        2      0   
 457225   9981903  17.0    0   0      1        0        0        2      1   
 457226   9982950  45.0    0   0      1        0        0        2      0   
 
        PL_CBSA  ...   NPR    TOTCHG PROCTYPE    DX1 ASCHED AWEEK

In [16]:
self.make_twin_agg_query_w_aggfntn('sample_s_b','sample_s_l')

{'query_real':           PNUM_R   AGE  DIED   DNR FEMALE HOSPBRTH Homeless HISPANIC NEOMAT  \
 0        7452218  47.0  None  None   None        0        0     None      0   
 1        7452218  47.0  None  None   None        0        0     None      0   
 2       12877535   NaN     0     0   None        0        0     None      0   
 3       13412288  26.0     0     0   None        0        0     None      0   
 4        1362038  22.0     0     0   None        0        0     None      0   
 ...          ...   ...   ...   ...    ...      ...      ...      ...    ...   
 457222  13892415   0.0     0     0      0        0        0        2      0   
 457223  11919360  42.0     0     0      1        0        0        2      0   
 457224   4812735  88.0     0     0      1        0        0        2      0   
 457225   6207339  27.0     0     0      1        0        0        2      1   
 457226   6207339  27.0     0     0      1        0        0        2      1   
 
        PL_CBSA  ...  NP